<a href="https://colab.research.google.com/github/anandababugudipudi/Boston-House-Prices-GD-Linear-Regression/blob/main/Boston_House_Prices_GD_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Boston House Prices Prediction using Linear Regression using GD**

###**Linear Regression** 
Linear Regression is a predictive modeling technique that finds a relationship between independent variable(s) and dependent variable(s). The independent variable(iv)’s can be categorical or continuous, while dependent variable(dv)s are continuous. Underlying function mapping iv’s and dv’s can be linear, quadratic, polynomial or other non-linear functions(like sigmoid function in logistic regression).

> Regression techniques are heavily used in making real estate price prediction, financial forecasting, predicting traffic arrival time (ETA).

###**Importing the necessary packages**

In [ ]:
import numpy as np
import pandas as pd
import torch 
import matplotlib.pyplot as plt

###**Importing the Dataset**
The Boston House prices dataset consists of 506 samples with 13 features with prices ranging from 5.0 to 50.0. Each record in the database describes a Boston suburb or town. The data was drawn from the Boston Standard Metropolitan Statistical Area (SMSA) in 1970. The attributes are deﬁned as follows (taken from the UCI Machine Learning Repository


In [ ]:
col_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
# Reading the csv file into numpy array
data = pd.read_csv('./boston-house-prices.csv', header = None, delimiter = r"\s+", names = col_names)
data.head()

In [ ]:
# Separating the features and labels in the dataset
features = data.drop('MEDV', axis = 1)
labels = data['MEDV']
print(features)
print(labels)

        CRIM    ZN  INDUS  CHAS    NOX  ...  RAD    TAX  PTRATIO       B  LSTAT
0    0.00632  18.0   2.31     0  0.538  ...    1  296.0     15.3  396.90   4.98
1    0.02731   0.0   7.07     0  0.469  ...    2  242.0     17.8  396.90   9.14
2    0.02729   0.0   7.07     0  0.469  ...    2  242.0     17.8  392.83   4.03
3    0.03237   0.0   2.18     0  0.458  ...    3  222.0     18.7  394.63   2.94
4    0.06905   0.0   2.18     0  0.458  ...    3  222.0     18.7  396.90   5.33
..       ...   ...    ...   ...    ...  ...  ...    ...      ...     ...    ...
501  0.06263   0.0  11.93     0  0.573  ...    1  273.0     21.0  391.99   9.67
502  0.04527   0.0  11.93     0  0.573  ...    1  273.0     21.0  396.90   9.08
503  0.06076   0.0  11.93     0  0.573  ...    1  273.0     21.0  396.90   5.64
504  0.10959   0.0  11.93     0  0.573  ...    1  273.0     21.0  393.45   6.48
505  0.04741   0.0  11.93     0  0.573  ...    1  273.0     21.0  396.90   7.88

[506 rows x 13 columns]
0      24.0
1  

In [ ]:
# Converting the pandas dataframes into PyTorch Tensors
# From here onwards our features are inputs and labels are targets
inputs = torch.tensor(features.values).float()
targets = torch.tensor(labels.values).float()
print(inputs)
print(targets)

In [ ]:
# Shape of inputs and targets
print(f"Shape of inputs is {inputs.shape}")
print(f"Shape of labels is {targets.shape}")

###**Linear Regression Model from scratch**

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [ ]:
# Weights and Biases
w = torch.randn(506, 13, requires_grad = True)
b = torch.randn(506, requires_grad = True)
print(w)
print(b)

Our model is simply a function that performs a matrix multiplication of the `features` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

> **X** `x` **W**<sup>T</sup> + **b**

We can define the model as follows:

In [ ]:
# Define the model
def model(x):
  return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [ ]:
# Generate the predictions on our inputs
preds = model(inputs)
print(preds)

Let's compare the predictions of our model with the actual targets.

In [ ]:
# Compare with targets
print(targets)

There is a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

###**Loss function**

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [ ]:
# Define a function to find MSE loss
def mse(t1, t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [ ]:
# Let's compute the loss
loss = mse(preds, targets)
print(loss)

Interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range of 5 - 50. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

###**Compute gradients**

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [ ]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [ ]:
# Gradients for weights
print(w)
print(w.grad)

In [ ]:
# Gradients for bias
print(b)
print(b.grad)

###**Adjust weights and biases to reduce the loss**

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [ ]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [ ]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [ ]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

###**Train the model using gradient descent**

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [ ]:
# 1. Generate predictions
preds = model(inputs)
print(preds)

In [ ]:
# 2. Calculate the loss
loss = mse(preds, targets)
print(loss)

In [ ]:
# 3. Compute gradients w.r.t. weights and biases
loss.backward()
print(w.grad)
print(b.grad)

In [ ]:

with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [ ]:
# New gradients and biases
print(w)
print(b)

With the new weights and biases, the model should have a lower loss.

In [ ]:
# Calculate the loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

We can alredy observe the change in the loss.

###**Train for multiple epochs**

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 1000 epochs.

In [ ]:
# Train for 1000 epochs
for i in range(1000):
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

# Calculate the loss once agian
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

#**Let's put all the code together**

In [ ]:
# Importing the necessary packages
import numpy as np
import pandas as pd
import torch 
import math
import matplotlib.pyplot as plt

# Importing the Dataset
col_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
# Reading the csv file into numpy array
data = pd.read_csv('./boston-house-prices.csv', header = None, delimiter = r"\s+", names = col_names)

# Separating the features and labels in the dataset
features = data.drop('MEDV', axis = 1)
labels = data['MEDV']

# Converting the pandas dataframes into PyTorch Tensors
# From here onwards our features are inputs and labels are targets
inputs = torch.tensor(features.values).float()
targets = torch.tensor(labels.values).float()

# Shape of inputs and targets
print(f"Shape of inputs is {inputs.shape}")
print(f"Shape of labels is {targets.shape}")

# Weights and Biases
w = torch.randn(506, 13, requires_grad = True)
b = torch.randn(506, requires_grad = True)

# Define the model
def model(x):
  return x @ w.t() + b

# Define a function to find MSE loss
def mse(t1, t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()

# Train the model using Gradient Descent
'''
We can train the model using the following steps:
1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero
'''
def fit(inputs, targets, epochs, w, b):
  # Train for n epochs
  for epoch in range(epochs):
    # 1. Generate predictions
    preds = model(inputs)

    # 2. Calculate the loss
    loss = mse(preds, targets)

    # 3. Compute gradients w.r.t. weights and biases
    loss.backward()

    # 4. Adjust weights and 5. reset gradients
    with torch.no_grad():
      w -= w.grad * 1e-5
      b -= b.grad * 1e-5
      w.grad.zero_()
      b.grad.zero_()
    # Print the progress
    if (epoch + 1) % 100 == 0:
      print(f"Epoch [{epoch+1}/{epochs}], Loss : {round(loss.item(),4)}")
# Initially we are training the model with 1000 epochs
epochs = 1000
fit(inputs, targets, epochs, w, b)
model(inputs[0])